In [0]:
#May have a memory crash in CPU or GPU mode. If so, run in TPU mode. 

#This notebook downloads the FFNN embedding files directly from my google drive 

#run all cells. The cells at the bottom creates a CSV file

In [0]:
import pandas as pd
import numpy as np
import os
import csv
from tqdm import tqdm


!pip install -U -q PyDrive

from google.colab import files
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# drive.mount('/content/gdrive')
# files = os.listdir("/content/gdrive/My Drive/FFNNEmbeds")

    100% |████████████████████████████████| 993kB 20.5MB/s 

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.



In [0]:
import requests

In [0]:
def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

In [0]:
if not os.path.exists('filez'):
    os.makedirs('filez')

In [0]:
file_id = '1hbk2z_zwTwKPZ3yKyVjjvSaj5rbnmBmj'

download_file_from_google_drive(file_id, 'filez/eHealthForums.csv')

In [0]:
file_id = '1rUU6wrEW72dbC3D7yJB1G5QdUg9UImS6'

download_file_from_google_drive(file_id, 'filez/healthTapFFNNEmbeddings.csv')

In [0]:
file_id = '1vsLTKh6A7Tzpyf_qHHYgYG4wzX71ThoF'

download_file_from_google_drive(file_id, 'filez/askDocsFFNNEmbeddings.csv')

In [0]:
file_id = '1y6LzAZDhNwMtnz6QFUwMWKENzayvzeUF'

download_file_from_google_drive(file_id, 'filez/webMDFFNNEmbeddings.csv')

In [0]:
file_id = '15x44Pglx3bg46Bqp2HaWR1OW3LFOURXr'

download_file_from_google_drive(file_id, 'filez/iClinicFFNNEmbeddings.csv')

In [0]:
os.listdir('filez')

['eHealthForums.csv',
 'healthTapFFNNEmbeddings.csv',
 'askDocsFFNNEmbeddings.csv',
 'webMDFFNNEmbeddings.csv',
 'iClinicFFNNEmbeddings.csv']

In [0]:
files = os.listdir('filez')

In [0]:
files

['eHealthForums.csv',
 'healthTapFFNNEmbeddings.csv',
 'askDocsFFNNEmbeddings.csv',
 'webMDFFNNEmbeddings.csv',
 'iClinicFFNNEmbeddings.csv']

In [0]:
def fix_array(x):
    x = np.fromstring(
    x.replace('\n','')
    .replace('[','')
    .replace(']','')
    .replace('  ',' '), sep=' ')
    return x.reshape((1, 768))

In [0]:
pd.options.display.max_rows = 700
pd.set_option('expand_frame_repr', True)
pd.set_option('max_colwidth', 250)
pd.get_option("display.max_rows")

700

In [0]:
qa = pd.read_csv("filez/" + files[0])
for file in files[1:]:
    print(file)
    qa = pd.concat([qa, pd.read_csv("filez/" + file)], axis = 0)

healthTapFFNNEmbeddings.csv
askDocsFFNNEmbeddings.csv
webMDFFNNEmbeddings.csv
iClinicFFNNEmbeddings.csv


In [0]:
del qa['Unnamed: 0']

In [0]:
qa["Q_FFNN_embeds"] = qa["Q_FFNN_embeds"].apply(fix_array)
qa["A_FFNN_embeds"] = qa["A_FFNN_embeds"].apply(fix_array)

In [0]:
# qa = qa.sample(frac = 1)
# qa.reset_index(inplace = True, drop = True)

In [0]:
qa = qa.reset_index(drop=True)

In [0]:
qa.head()

,question,answer,Q_FFNN_embeds,A_FFNN_embeds
0,how do i stop smoking now,stopping smoking is about will power and being steadfast. you can stop safely by having bupropion or nicotine patch cover initially in consult with a doctor. contact an addiction clinic near you. wishing you best of health thanks,"[[0.198553607, 0.238654211, 0.0252966508, 0.536100566, -0.270931244, 0.143292472, 0.66546452, -0.364487559, 0.831103981, -0.257745147, 1.03932917, 0.0567179397, -0.173779696, 0.332652003, -0.152597591, -0.0945576429, -0.0650781095, 0.41572243, 0....","[[-0.107524604, 0.131869704, -0.135342821, 0.236110285, 0.144869864, 0.125502259, 0.0284669325, -0.241687, 0.359113157, 0.215388954, 0.0614173561, -0.117674321, -0.18703939, 0.422599256, 0.291348606, 0.132292926, -0.0371215381, -0.328066021, 0.38..."
1,i had a tubaligation 4 years ago and also have a minor case of endometriosis i am only 27 and my period stopped completely 3 months ago no more spotting in between nothing i am not stressed no change in daily habits diet weight nothing all urine ...,hello this sounds quite unfamiliar that due to no reason at this age you have no cycle for last three months. pregnancy is surely a remote possibility as there was a tubal ligation. the endometriosis is also unlikely could cause stoppage of cycle...,"[[-0.0332615711, 0.0908259526, 0.12959592, 0.188717857, 0.0553386658, -0.138015583, 0.404416293, -0.306202292, -0.27767241, 0.224860445, 1.30278707, 0.0921037346, -0.0639316812, 1.31605291, 0.228724718, -0.043722976, 0.0802280307, 0.422940314, 0....","[[-0.157639518, 0.194728285, 0.0597435087, 0.103870191, 0.252210379, 0.0174597353, 0.201248869, -0.253417432, -0.478795975, 0.379764229, -0.190171927, 0.266052544, 1.04273164, 0.483350039, 0.188350126, 0.873891115, 0.110315099, -0.244599178, 0.17..."
2,could extra caffeine consumption be a cause of mild unilateral breast tenderness?,extra caffeine can cause gastric discomfort the pain of which can be felt at one side of breast mostly left side but right also is possible and some times whole chest. get evaluated for any local pathology like mastitis. i hope it helps. stay in ...,"[[-0.00937359221, 0.218749091, 0.00305952434, 0.443503141, 1.12520206, -0.112999737, 1.00371063, 0.0340681076, 0.292715639, 0.170468584, 1.10793352, 0.0736165494, 0.077357851, -0.298604459, 0.114351489, -0.0838725939, 0.839284778, 1.59145796, 0.2...","[[0.159728408, 0.20174022, 0.047992982, 0.0541549884, 0.538674176, -0.00781832263, -0.068097882, -0.111649357, -0.370063186, 0.439989567, -0.090130806, 0.162251547, -0.00463378336, 0.120588772, 0.182921246, 0.762594581, -0.260490298, -0.395742387..."
3,"hello- i am a 24 year old female 5""4 & 115 lb & was diagnosed with pcos at 19. i've always been active & extremely healthy. spironolactone & an iud helped with acne & pain but in 2013 i had an ovarian torsion requiring surgery & they saved my rig...",hello thanks for submitting your question here. as per the description you have undergone repeated surgery for ovarian torsion where the ovary is preserved. this sounds a bit different as in torsion it is a dictum to remove the whole ovary. now a...,"[[-0.062063057, 0.178399101, 0.0026178658, 0.271553308, 0.306988388, -0.30831778, 0.0854902267, -0.268227369, 0.141994387, 0.482396036, 0.377293408, -0.00115496782, -0.0811023712, 0.627834022, 0.373512119, -0.101564594, -0.0333173573, 0.86133039,...","[[-0.13206917, 0.218641087, -0.142680183, 0.093027018, 0.518154204, -0.0593009144, -0.154638484, -0.39215523, -0.386736423, 0.185574383, -0.0550778024, 0.0291010961, 1.04651797, 0.106718101, 0.237218335, 0.55746001, -0.12029212, -0.188211352, 0.7..."
4,"i was wanting to know if you could tell me if its possible for a woman to go through all their pregnancy without knowing they are pregnant and have periods and ngative pregnancy tests? iwas watching the show called ""i didnt know i was pregnant"" a...",i am glad to help you out. this is not possible scientifically for a woman not to kn

In [0]:
question_bert = np.concatenate(qa["Q_FFNN_embeds"].values, axis=0)
answer_bert = np.concatenate(qa["A_FFNN_embeds"].values, axis=0)

In [0]:
answer_bert.shape


(239402, 768)

In [0]:
question_bert = question_bert.astype('float32')
answer_bert = answer_bert.astype('float32')

In [0]:
faiss.normalize_L2(question_bert)
faiss.normalize_L2(answer_bert)

In [0]:
#just the CPU version of FAISS, will have to look deeper on how to get GPU version, but works fast enough for now 

!wget  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!tar xvjf faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/
!pip install mkl

--2019-05-01 12:59:40--  https://anaconda.org/pytorch/faiss-cpu/1.2.1/download/linux-64/faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2
Resolving anaconda.org (anaconda.org)... 104.17.92.24, 104.17.93.24, 2606:4700::6811:5c18, ...
Connecting to anaconda.org (anaconda.org)|104.17.92.24|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://binstar-cio-packages-prod.s3.amazonaws.com/5a15c9c5c376961204909d87/5aa7f0a65571b411e5c259be?response-content-disposition=attachment%3B%20filename%3D%22faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2%22%3B%20filename%2A%3DUTF-8%27%27faiss-cpu-1.2.1-py36_cuda9.0.176_1.tar.bz2&response-content-type=application%2Fx-tar&X-Amz-Algorithm=AWS4-HMAC-SHA256&X-Amz-Expires=60&X-Amz-Date=20190501T125940Z&X-Amz-SignedHeaders=host&X-Amz-Security-Token=AgoJb3JpZ2luX2VjEKH%2F%2F%2F%2F%2F%2F%2F%2F%2F%2FwEaCXVzLWVhc3QtMSJGMEQCIBkt9FKvm8uQXf47qGCNDjttHGN5C5rpeUE%2FekAkiKW7AiA3xCWwarn1ywD2%2BJeG5tvAjlk0lTi%2BzO5DgTdg3WZ8pyrjAwiK%2F%2F%2F%2F%2F%2F%2F

In [0]:
import faiss

Failed to load GPU Faiss: No module named 'faiss.swigfaiss_gpu'
Faiss falling back to CPU-only.


In [0]:

answer_index = faiss.IndexFlatIP(answer_bert.shape[-1])
answer_index.add(answer_bert)

question_index = faiss.IndexFlatIP(question_bert.shape[-1])
question_index.add(question_bert)

In [0]:
def topKforGPT2(xf, kf):
    D1, I1 = answer_index.search(question_bert[xf:xf+1].astype('float32'), kf)
    D2, I2 = question_index.search(question_bert[xf:xf+1].astype('float32'), kf)
    return I1, I2

In [0]:

numberSamples=10

output = open('GPT2data.csv', "w")
writer = csv.writer(output)

firstrow = ['question', 'answer']
for ii in range(0, numberSamples):
    firstrow.append('question'+str(ii))
    firstrow.append('answer'+str(ii))

writer.writerow(firstrow)

# for k in tqdm(range(1000), mininterval=30, maxinterval=60):
for k in tqdm(range(qa.shape[0]), mininterval=30, maxinterval=60):
    rowfill=[]
    rowfill.append(qa["question"][k])
    rowfill.append(qa["answer"][k])
    aa, qq = topKforGPT2(k, int(numberSamples/2))
    aaa = qa.loc[list(aa[0]), :]
    qqq = qa.loc[list(qq[0]), :]
    aaaa =  [*sum(zip( list(aaa['question']) ,list(aaa['answer'])  ),())]
    qqqq = [*sum(zip( list(qqq['question']) ,list(qqq['answer'])  ),())]  
    finalfill=aaaa+qqqq
    rowfill = rowfill + finalfill
    writer.writerow(rowfill)
    auth.authenticate_user()
    gauth = GoogleAuth()
    gauth.credentials = GoogleCredentials.get_application_default()
    drive = GoogleDrive(gauth)
output.close()

  0%|          | 0/239402 [00:00<?, ?it/s]


KeyboardInterrupt: ignored

In [0]:
output.close()

In [0]:
478.804

In [0]:
uploadModel = drive.CreateFile() 
uploadModel.SetContentFile('GPT2data.csv')
uploadModel.Upload()